# Spam Classifier

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
spam_tokenizer = AutoTokenizer.from_pretrained("AntiSpamInstitute/spam-detector-bert-MoE-v2.2")
spam_model = AutoModelForSequenceClassification.from_pretrained("AntiSpamInstitute/spam-detector-bert-MoE-v2.2")

# Sample text
texts = [
    "Congratulations! You've won a $1,000 Walmart gift card. Click here to claim now.",
    "Hey, are we still meeting for lunch today?"
]

# Tokenize the input
inputs = spam_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Get model predictions
with torch.no_grad():
    outputs = spam_model(**inputs)
    logits = outputs.logits

# Apply softmax to get probabilities
probabilities = torch.softmax(logits, dim=1)

# Get predicted labels
predictions = torch.argmax(probabilities, dim=1)

# Map labels to class names
label_map = {0: "Not Spam", 1: "Spam"}
for text, prediction in zip(texts, predictions):
    print(f"Text: {text}\nPrediction: {label_map[prediction.item()]}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Congratulations! You've won a $1,000 Walmart gift card. Click here to claim now.
Prediction: Spam

Text: Hey, are we still meeting for lunch today?
Prediction: Not Spam



# Sentiment Analyser

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess links (link placeholders)
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        #t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
sentiment_model = f"cardiffnlp/twitter-roberta-base-{task}"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model)

# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model)
#sentiment_model.save_pretrained(sentiment_model)

text = "Good night 😊"
text = preprocess(text)
encoded_input = sentiment_tokenizer(text, return_tensors='pt')
output = sentiment_model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) positive 0.8466
2) neutral 0.1458
3) negative 0.0076


# Sarcasm Detector

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess links (link placeholders)
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        #t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='irony'
irony_model = f"cardiffnlp/twitter-roberta-base-{task}"
irony_tokenizer = AutoTokenizer.from_pretrained(irony_model)

# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
irony_model = AutoModelForSequenceClassification.from_pretrained(irony_model)
#sentiment_model.save_pretrained(sentiment_model)

text = "wow youre so good at studying, i couldnt even study that hard and still not pass"
text = preprocess(text)
encoded_input = irony_tokenizer(text, return_tensors='pt')
output = irony_model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) irony 0.8759
2) non_irony 0.1241


# Preprocessing

In [4]:
import pandas as pd

# Load your CSV
df = pd.read_csv("final_eval_dataset_reviews.csv", encoding='latin1')

# Apply preprocessing to each review
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '<link>' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Create a new column for preprocessed reviews
df['preprocessed_review'] = df['text'].apply(preprocess)

print(df[['text', 'preprocessed_review']].head())


df.to_csv("processed_reviews.csv", index=False)

                                                text  \
0  Big supermarket here with two storey. Can find...   
1  Tried to collect book bugs. Before 5pm the lib...   
2  Oh, my God, it's just a wavey. Wooden bridge w...   
3  No sheltered pathway out of the office buildin...   
4                   hard to find and so confusing...   

                                 preprocessed_review  
0  Big supermarket here with two storey. Can find...  
1  Tried to collect book bugs. Before 5pm the lib...  
2  Oh, my God, it's just a wavey. Wooden bridge w...  
3  No sheltered pathway out of the office buildin...  
4                   hard to find and so confusing...  


# Run Classifiers

In [5]:
import pandas as pd

# Function to run all classifiers
def run_classifiers(preprocessed_text):
    results = {}
    try:
        # Spam
        inputs = spam_tokenizer(preprocessed_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = spam_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).numpy()[0]
        pred = np.argmax(probs)
        results['spam_label_'] = "Spam" if pred == 1 else "Not Spam"

        # Sentiment
        inputs = sentiment_tokenizer(preprocessed_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = sentiment_model(**inputs)
        scores = softmax(outputs.logits.detach().numpy()[0])
        pred = np.argmax(scores)
        labels_list = ['Negative', 'Neutral', 'Positive']
        results['sentiment_label_'] = labels_list[pred]

        # Sarcasm
        inputs = irony_tokenizer(preprocessed_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = irony_model(**inputs)
        scores = softmax(outputs.logits.detach().numpy()[0])
        pred = np.argmax(scores)
        results['sarcasm_label_'] = "Sarcastic" if pred == 1 else "Not Sarcastic"

    except Exception as e:
        # fallback if model fails
        results['spam_label_'] = 'Error'
        results['sentiment_label_'] = 'Error'
        results['sarcasm_label_'] = 'Error'

    return pd.Series(results)

# Apply to your DataFrame
df[['spam_label_', 'sentiment_label_', 'sarcasm_label_']] = df['preprocessed_review'].apply(run_classifiers)

# Check
print(df[['spam_label_', 'sentiment_label_', 'sarcasm_label_']].head())

df.to_csv("processed_reviews.csv", index=False)


  spam_label_ sentiment_label_ sarcasm_label_
0    Not Spam         Negative  Not Sarcastic
1    Not Spam         Negative  Not Sarcastic
2    Not Spam         Positive  Not Sarcastic
3    Not Spam         Negative  Not Sarcastic
4    Not Spam         Negative  Not Sarcastic


# Relevancy Scoring

In [6]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("processed_reviews.csv")
nlp = spacy.load("en_core_web_lg")

def compute_relevance(row):
    review = row['preprocessed_review']
    categories = row['categoryName']  # column in your CSV
    types = row['types']              # column in your CSV
    
    types_in_array = [x.strip().replace("_", " ") for x in types.split(",")]
    types_in_array.append(categories)
    types_in_array = [t for t in types_in_array if t not in ['point of interest', 'establishment']]
    
    word_vec = sum([nlp(k).vector for k in types_in_array]) / len(types_in_array)
    word_vec = word_vec.reshape(1, -1)
    
    doc = nlp(review)
    score = sum([cosine_similarity(word_vec, sent.vector.reshape(1,-1))[0][0] for sent in doc.sents])
    avg_score = score / len(list(doc.sents)) if len(list(doc.sents)) != 0 else 0
    
    relevance_threshold = 0.3
    label = "Relevant" if avg_score >= relevance_threshold else "Not Relevant"
    
    return pd.Series([avg_score, label])

df[['relevance_score', 'relevance_label']] = df.apply(compute_relevance, axis=1)
df.to_csv("processed_reviews1.csv", index=False)


# Run Summariser

In [21]:
import requests

def ollama_summarize(text, category,max_words=100, model="llama3.2"):
    prompt = f"""
            You are a review summarizer. 
            Your task is to read a user review and generate a short, concise summary that captures the main points and meaning 
            without losing important details. 
            
            Use clear and simple language. 
            Keep the summary brief, ideally two or three sentences. 
            Do not add opinions or extra information not present in the review. 

            If the review seems negative, include the word negative in the summary.
            If the review seems like an average experience, use the word average in the summary.
            If the review seems like a positive experience, use the word positive in the summary.
            If the review does not describe the location itself, its services, facilities, or the user’s experience there → Output: Briefly explain why the review is irrelevant, and include the word irrelevant in the sentence.
            If the review is too promotional/advertisement-like to a brand or product→ Output: Summarise the promotional content and explain why it sounds promotional and include the word advertisement-like in the sentence.
            If the review explicitly discriminates against certain groups or races → Output: Identify the group being discriminated against and use the word insensitive in the sentence.
            If the review is informative (about the location) → Output: Summarise the main points of information given in 1–2 sentences, and use the word informative in the sentence.
            If the reviewer seems like they did not go to the location before → Output: Explain why it seems the reviewer did not go and use the phrase did not visit in the sentence.
            
            You are given some context to the user review, including whether sarcasm is present. 
            If sarcasm is detected, interpret the review accordingly and summarize the intended meaning rather than the literal words.
            
            Summarize the following text in at most {max_words} words. 
            
            Keep the reply in this format:
            1) Location Type 
            2) Summarised Review
            
            Here are some examples:
            
            User: Location: MRT Station "On my way to my teacher's event, just had lunch before I left"  
            Model:  
            1) MRT Station  
            2) The review discusses personal activities and lunch, which are irrelevant to the MRT station experience.
            
            User: Location : Store "Get the new sleek and trendy sunglasses today at sunglaz"
            Model:
            1) Store
            2) The review sounds very promotional and advertisement-like for sunglaz sunglasses
            
            User: Location : Tourist Attraction "There is just too many foreigners here nowadays, everyone should go back to where they came from."
            Model:
            1) Tourist Attraction
            2) The reviewer is insensitive and discriminates against foreigners visiting the tourist attraction.
            
            User: Location : Hotel "The hotel was average with okay service. Staff were friendly and the breakfast buffet had a lot of options. It is close to many bus interchanges and mrt stations"
            Model:
            1) Hotel
            2) Average hotel with okay service, friendly staff ,and a wide breakfast buffet. The reviewer also gives informative facts that the hotel is also close to many bus interchanges and mrt stations.
            
            User: Location : Tourist Attraction "I came here and it was closed, my friends mentioned that the place is smelly though I have never been"
            Model:
            1) Tourist Attraction
            2) The reviewer came when the attraction was closed, and did not enter. The reviewer also mentioned that they did not visit the attraction.
            
            User: "This park is very peaceful and clean. A great place to walk and relax during my lunch break."
            Model:
            1) Park
            2) The reviewer had a positive experience with the cleanliness and peacefulness of the park. They like to relax in the park during their lunch break.
            
            User: "The restaurant food was cold and service was slow. Not worth the money."
            Model:
            1) Restaurant
            2) The reviewer had a negative experience with the cold food and slow service from the restaurant, and notes that the experience was not worth the money.
            
            User : Location : Cafe (Sarcastic) "Oh great, another long line at the cafe. Just what I needed today."
            Model:
            1) Cafe
            2) The reviewer had a negative experience with the long line at the cafe.

            User : Location : Restaurant "The food was okay, and the restaurant was not so crowded"
            Model:
            1) Cafe
            2) The reviewer had an average experience and was neutral about the experience because the food was average.
            
            Now summarize this review: 
            Do not add opinions or extra information not present in the review.
            Location : {category} "{text}"
            """

    r = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False},
        timeout=300
    )
    r.raise_for_status()
    return r.json()["response"].strip()

In [ ]:
import pandas as pd
from tqdm import tqdm

# Config
batch_size = 1  # number of reviews per batch

# Load CSV
df = pd.read_csv("processed_reviews1.csv")

# Function to build modified review text
def build_modified_review(row):
    text = row['preprocessed_review']

    # Add sarcasm hint
    if row.get('sarcasm_label_') == "Sarcastic":
        text += " (Sarcastic)"

    text += f" Category: {row.get('categoryName', 'Unknown')}"

    return text

# Function to append labels to summary
def append_labels(row, summary):
    labels = []

    if row.get('sentiment_label_'):
        labels.append(row['sentiment_label_'])

    if row.get('spam_label_') == "Spam":
        labels.append("Spam")

    if row.get('sarcasm_label_') == "Sarcastic":
        labels.append("Sarcastic")

    if row.get('relevance_label') == "Not Relevant":
        labels.append("Irrelevant")

    if labels:
        return summary + " | Labels: " + ", ".join(labels)
    return summary

# Build modified reviews column
df['modified_review'] = df.apply(build_modified_review, axis=1)

# Summarize in batches
all_summaries = []

for i in tqdm(range(0, len(df), batch_size)):
    batch = df.iloc[i:i+batch_size]

    # Prepare batch lists
    batch_texts = batch['modified_review']#.tolist() since batch size = 1 we dont neeed list
    batch_categories = batch['categoryName']#.tolist()

    # Summarize all reviews in the batch at once (if supported)
    batch_summaries = ollama_summarize(batch_texts, batch_categories)
    print(batch_texts)
    print(batch_summaries)
    print("\n")

    # Append labels to each summary
    for j, row in batch.iterrows():
        summary_with_labels = append_labels(row, batch_summaries)
        print(summary_with_labels)
        all_summaries.append(summary_with_labels)

# Add final summaries to dataframe
df['summary'] = all_summaries

# Save to a new CSV
df.to_csv("processed_reviews2.csv", index=False)

print("Summarization complete! Saved to processed_reviews.csv")



  0%|▎                                                                                 | 1/314 [00:02<15:18,  2.93s/it]

0    Big supermarket here with two storey. Can find...
Name: modified_review, dtype: object
1) Grocery Store
2) The reviewer found a big supermarket with two floors, but the review is too long and incomplete to summarize its contents accurately, making it irrelevant.


1) Grocery Store
2) The reviewer found a big supermarket with two floors, but the review is too long and incomplete to summarize its contents accurately, making it irrelevant. | Labels: Negative


  1%|▌                                                                                 | 2/314 [00:05<15:07,  2.91s/it]

1    Tried to collect book bugs. Before 5pm the lib...
Name: modified_review, dtype: object
1) Public library
2) The reviewer mentions trying to collect book bugs and mentions a specific time frame (before 5pm), which is informative about their experience at the library.


1) Public library
2) The reviewer mentions trying to collect book bugs and mentions a specific time frame (before 5pm), which is informative about their experience at the library. | Labels: Negative


  1%|▊                                                                                 | 3/314 [00:08<14:47,  2.86s/it]

...

76    The ArtScience Museum gave me one of the most ...
Name: modified_review, dtype: object
1) Museum
2) The reviewer had a positive experience with the museum, stating it provided them with one of the most memorable experiences.


1) Museum
2) The reviewer had a positive experience with the museum, stating it provided them with one of the most memorable experiences. | Labels: Positive


 25%|████████████████████                                                             | 78/314 [03:36<11:07,  2.83s/it]

77    While planning my trip to Singapore, I heard a...
Name: modified_review, dtype: object
1) Museum
2) The reviewer mentions that they heard about the museum while planning their trip to Singapore but does not provide any personal experience or information about the museum itself.


1) Museum
2) The reviewer mentions that they heard about the museum while planning their trip to Singapore but does not provide any personal experience or information about the museum itself. | Labels: Positive


 25%|████████████████████▍                                                            | 79/314 [03:38<10:59,  2.81s/it]

78    Great food. It was packed. The seating and amb...
Name: modified_review, dtype: object
1) Cafe
2) The reviewer had a positive experience with great food at a busy cafe with good seating and amenities.


1) Cafe
2) The reviewer had a positive experience with great food at a busy cafe with good seating and amenities. | Labels: Positive


# Cosine Similarity Algorithm

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def compute_relatedness_local(review_text,type_context):
    # Create enhanced description
    # Get embeddings locally
    review_emb = model.encode([review_text])
    type_emb = model.encode([type_context])

    # Compute similarity
    similarity = cosine_similarity(review_emb, type_emb)[0][0]
    return similarity

In [ ]:
negative_review = (
    "This review describes a negative experience at a location. "
    "It focuses on problems, dissatisfaction, or poor service. "
    "Negative experiences include rude or unhelpful staff, long waits, unclean facilities, broken equipment, or poor service. "
    "Examples: "
    "'The parking attendant was rude and unhelpful.', "
    "'The restaurant was dirty and the food was cold.', "
    "'I waited 45 minutes for my order and the staff ignored me.'"
)

neutral_review = (
    "Informative"
    "This review reflects a neutral or average experience at a location. "
    "It reports factual observations about the place, its services, facilities, or environment without showing clear satisfaction or dissatisfaction. "
    "Examples: "
    "'The café has tables and chairs arranged neatly.', "
    "'There is parking available near the building.', "
    "'Staff responded to my questions as expected.', "
    "'The mall contains a variety of shops and is straightforward to navigate.'"
    "'The hotel is located downtown. The rooms were average, and check-in was standard.'"
)

advertisement = (
    "This review contains promotional content that are advertisement-like, advertising, or links. "
    "It includes attempts to sell products or services, provide discounts, or promote websites. "
    "Examples: "
    "'Best pizza! Visit www.pizzapromo.com for discounts!', "
    "'Buy cheap sunglasses at www.freesunglasses.com!', "
    "'Order now and receive free shipping!'"
    "'Craving the best burgers in town? Visit McDonald's today and taste the magic – juicy, flavorful, unforgettable!'"
)

irrelevant_content = (
    "This review contains content unrelated to the location. "
    "Irrelevant content includes personal topics, experiences elsewhere, or unrelated products. "
    "Examples: "
    "'I love my new phone, but this place is too noisy.', "
    "'The movie I watched yesterday was amazing, but the restaurant was okay.', "
    "'My vacation in Bali was fantastic, but the mall was just average.'"
)

rant_without_visit = (
    "This review contains complaints or rants from someone who has not personally visited the location. "
    "Examples: "
    "I have never visited but I heard that its good"
    "My family said that the place was awesome"
    "'Never been here, but I heard it’s terrible.', "
    "'Someone told me the staff is rude, I wouldn’t go there.', "
    "'I read online that the service is bad, so avoid it.'"
)

positive_review = (
    "This review expresses a positive experience at a location. "
    "It highlights aspects of the place, its services, facilities, or environment that the reviewer enjoyed or appreciated. "
    "Examples: "
    "'The café had a friendly staff and the coffee was excellent.', "
    "'Parking was easy to find and convenient.', "
    "'Staff went out of their way to answer my questions and provide assistance.', "
    "'The mall has a great variety of shops and is easy to navigate.', "
    "'The restaurant served fresh and delicious food with prompt service.', "
    "'The park is well-maintained, clean, and relaxing.', "
    "'The hotel room was spacious, clean, and comfortable.', "
    "'The museum had engaging exhibits and helpful staff.', "
    "'The spa provided a relaxing atmosphere and excellent treatments.', "
    "'The store had well-organized shelves and attentive staff.'"
)

political_and_social_insensitivity = (
    "This label is only for reviews that contain clearly offensive, discriminatory, or inappropriate content related to race, religion, gender, nationality, political beliefs, or social groups."
    "Do not assign this label for neutral mentions of ethnicity, nationality, or culture, or for minor personal opinions."
    "Only use it if the review is explicitly prejudiced, hostile, or discriminatory."
    "Examples: "
    "'The hotel is too crowded with certain racial groups; it’s not enjoyable.', "
    "'I can’t believe they allow the foreign worker community here; it’s unacceptable.', "
)

policies = {
    "negative_review": negative_review,
    "neutral_review": neutral_review,
    "positive_review": positive_review,
    "advertisement": advertisement,
    "irrelevant_content": irrelevant_content,
    "rant_without_visit": rant_without_visit,
    "political_and_social_insensitivity" : political_and_social_insensitivity
}

In [ ]:
def classify_policy(policies, review_text, processed_review):
    max_score = -1
    max_policy = None

    for policy_name, policy_context in policies.items():
        x = compute_relatedness_local(
            review_text,
            policy_context)
        y = compute_relatedness_local( # y is the output of the review summariser
            processed_review,
            policy_context)

        score = 0.3*x + 0.7*y # Add a weighted score

        print(policy_name, x, y, score)

        if score > max_score:
            max_score = score
            max_policy = policy_name

    return max_policy,max_score

# Evaluate

In [ ]:
df = pd.read_csv("processed_reviews2.csv")
df["predicted_label"]= ""
df["predicted_label_score"]= ""

for index, row in tqdm(df.iterrows()):
    true_label = row["policy_label"]
    review = row["text"]
    summary = row["summary"]
    output_policy, policy_score = classify_policy(policies,review,summary)
    df.at[index, "predicted_label"] = output_policy
    df.at[index, "predicted_label_score"] = policy_score

df.to_csv("ensemble_evaluation_final_modified_summary.csv", index=False, encoding="utf-8")

In [ ]:
asdf